### Finding Key Connectors

In [1]:
users = [
    { "id": 0, "name": "Hero" },
    { "id": 1, "name": "Dunn" },
    { "id": 2, "name": "Sue" },
    { "id": 3, "name": "Chi" },
    { "id": 4, "name": "Thor" },
    { "id": 5, "name": "Clive" },
    { "id": 6, "name": "Hicks" },
    { "id": 7, "name": "Devin" },
    { "id": 8, "name": "Kate" },
    { "id": 9, "name": "Klein" },
    { "id": 10, "name": "Jen" }
]

friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
               (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

In [2]:
# init empty lists, "friends"
for user in users:
    user["friends"] = []

In [3]:
for i, j in friendships:
    users[i]["friends"].append(users[j])
    users[j]["friends"].append(users[i])

In [4]:
def number_of_friends(user):
    """how many friend does _user_ have?"""
    return len(user["friends"])

total_connections = sum(number_of_friends(user) for user in users)
total_connections

24

In [5]:
avg_connections = total_connections / len(users)
avg_connections
# new one, Jen, comes

2.1818181818181817

In [6]:
num_friends_by_id = [(user["id"], number_of_friends(user)) for user in users]
sorted(num_friends_by_id, 
       key=lambda pair: pair[1], 
       reverse=True)

[(1, 3),
 (2, 3),
 (3, 3),
 (5, 3),
 (8, 3),
 (0, 2),
 (4, 2),
 (6, 2),
 (7, 2),
 (9, 1),
 (10, 0)]

### Data Scientist You May Know

In [7]:
def friends_of_friend_ids_bad(user):
    return [foaf["id"] 
            for friend in user["friends"] 
            for foaf in friend["friends"]]

In [8]:
# Bad, there are dups
friends_of_friend_ids_bad(users[0])

[0, 2, 3, 0, 1, 3]

In [9]:
from collections import Counter

def not_the_same(user, other_user):
    return user["id"] != other_user["id"]

def not_friends(user, other_user):
    return all(not_the_same(other_user, friend) 
               for friend in user["friends"])

def friends_of_friend_ids(user):
    # like 'the friend who know each other'
    return Counter(foaf["id"]
            for friend in user["friends"]
            for foaf in friend["friends"]
            if not_the_same(user, foaf)
            and not_friends(user, foaf))

In [13]:
friends_of_friend_ids(users[3])

Counter({0: 2, 5: 1})

In [14]:
interests = [
    (0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
    (0, "Spark"), (0, "Storm"), (0, "Cassandra"),
    (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
    (1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
    (2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Python"),
    (3, "statistics"), (3, "regression"), (3, "probability"),
    (4, "machine learning"), (4, "regression"), (4, "decision trees"),
    (4, "libsvm"), (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"),
    (5, "Haskell"), (5, "programming languages"), (6, "statistics"),
    (6, "probability"), (6, "mathematics"), (6, "theory"),
    (7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"),
    (7, "neural networks"), (8, "neural networks"), (8, "deep learning"),
    (8, "Big Data"), (8, "artificial intelligence"), (9, "Hadoop"),
    (9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

In [15]:
def data_scientists_who_like(target_interest):
    return [user_id
            for user_id, interest in interests
            if interest == target_interest]
# inefficient

In [16]:
# build indexes
from collections import defaultdict

user_ids_by_interest = defaultdict(list)
interests_by_user_id = defaultdict(list)

for user_id, interest in interests:
    user_ids_by_interest[interest].append(user_id)
    interests_by_user_id[user_id].append(interest)

In [17]:
# Find most intrests in common
# 1. Iterate over the user's interests
# 2. For each interest, iterate over the other users with that interest
# 3. Keep count of how many times we see each other user
def most_common_interest_with(user):
    return Counter(interested_user_id
                   for interest in interests_by_user_id[user["id"]]
                   for interested_user_id in user_ids_by_interest[interest]
                   if interested_user_id != user["id"])

In [18]:
most_common_interest_with(users[0])

Counter({1: 2, 5: 1, 8: 1, 9: 3})

### Salaries and Experience

In [19]:
salaries_and_tenures = [(83000, 8.7), (88000, 8.1),
                        (48000, 0.7), (76000, 6),
                        (69000, 6.5), (76000, 7.5),
                        (60000, 2.5), (83000, 10),
                        (48000, 1.9), (63000, 4.2)]

In [20]:
salary_by_tenure = defaultdict(list)

for salary, tenure in salaries_and_tenures:
    salary_by_tenure[tenure].append(salary)
    
salary_by_tenure
# meaningless

defaultdict(list,
            {0.7: [48000],
             1.9: [48000],
             2.5: [60000],
             4.2: [63000],
             6: [76000],
             6.5: [69000],
             7.5: [76000],
             8.1: [88000],
             8.7: [83000],
             10: [83000]})

In [21]:
# bucket the tenures
def tenure_bucket(tenure):
    if tenure < 2:
        return "less then two"
    elif tenure < 5:
        return "between two and five"
    else:
        return "more than five"



In [22]:
# group together
salary_by_tenure_bucket = defaultdict(list)

for salary, tenure in salaries_and_tenures:
    bucket = tenure_bucket(tenure)
    salary_by_tenure_bucket[bucket].append(salary)
    
salary_by_tenure_bucket

defaultdict(list,
            {'between two and five': [60000, 63000],
             'less then two': [48000, 48000],
             'more than five': [83000, 88000, 76000, 69000, 76000, 83000]})

In [23]:
average_salary_by_bucket = {
    tenure_bucket: sum(salaries) / len(salaries)
    for tenure_bucket, salaries in salary_by_tenure_bucket.items()
}
average_salary_by_bucket

{'between two and five': 61500.0,
 'less then two': 48000.0,
 'more than five': 79166.66666666667}

### Paid Accounts

In [24]:
# very rough model
def predict_paid_or_unpaid(years_experience):
    if years_experience < 3.0:
        return "paid"
    elif years_experience < 8.5:
        return "unpaid"
    else:
        return "paid"

### Topic of Interest

In [25]:
# Count the words
# 1. Lowercase each interest
# 2. Split into words
# 3. Count the results
words_and_counts = Counter(word
                           for user, interest in interests
                           for word in interest.lower().split())

In [26]:
for word, count in words_and_counts.most_common():
    if count > 1:
        print(word, count)

big 3
data 3
java 3
python 3
learning 3
hadoop 2
hbase 2
cassandra 2
scikit-learn 2
r 2
statistics 2
regression 2
probability 2
machine 2
neural 2
networks 2
